In [1]:
import pandas as pd
import numpy as np

In [3]:
# Note: The files are often separated by semi-colons ';' instead of commas
# encoding='latin-1' is often required for this specific dataset to handle special characters
books = pd.read_csv('book_dataset/Books.csv')
users = pd.read_csv('book_dataset/Users.csv')
ratings = pd.read_csv('book_dataset/Ratings.csv')

C:\Users\hungt\AppData\Local\Temp\ipykernel_15616\2097659975.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('book_dataset/Books.csv')


In [4]:
# 2. Rename columns: Replace '-' with '_'
books.columns = books.columns.str.replace('-', '_')
users.columns = users.columns.str.replace('-', '_')
ratings.columns = ratings.columns.str.replace('-', '_')

In [5]:
print(books.columns)

Index(['ISBN', 'Book_Title', 'Book_Author', 'Year_Of_Publication', 'Publisher',
       'Image_URL_S', 'Image_URL_M', 'Image_URL_L'],
      dtype='object')


In [6]:
print(users.columns)

Index(['User_ID', 'Location', 'Age'], dtype='object')


In [7]:
print(ratings.columns)

Index(['User_ID', 'ISBN', 'Book_Rating'], dtype='object')


In [8]:
#dropping last three columns containing image URLs which will not be required for analysis
books.drop(['Image_URL_S', 'Image_URL_M', 'Image_URL_L'],axis=1,inplace=True)

In [9]:
#checking data types of columns
books.dtypes

ISBN                   object
Book_Title             object
Book_Author            object
Year_Of_Publication    object
Publisher              object
dtype: object

In [10]:
books.head()

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [11]:
# 1. FIX THE WARNING: Create a true copy
books = books.copy()

# 2. Manual fixes (The code you provided)
books.loc[books.ISBN == '0789466953', 'Year_Of_Publication'] = 2000
books.loc[books.ISBN == '0789466953', 'Book_Author'] = "James Buckley"
books.loc[books.ISBN == '0789466953', 'Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '0789466953', 'Book_Title'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"

books.loc[books.ISBN == '078946697X', 'Year_Of_Publication'] = 2000
books.loc[books.ISBN == '078946697X', 'Book_Author'] = "Michael Teitelbaum"
books.loc[books.ISBN == '078946697X', 'Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '078946697X', 'Book_Title'] = "DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)"

books.loc[books.ISBN == '2070426769', 'Year_Of_Publication'] = 2003
books.loc[books.ISBN == '2070426769', 'Book_Author'] = "Jean-Marie Gustave Le Clézio"
books.loc[books.ISBN == '2070426769', 'Publisher'] = "Gallimard"
books.loc[books.ISBN == '2070426769', 'Book_Title'] = "Peuple du ciel, suivi de 'Les Bergers"


In [12]:
def missing_values(dataframe):
  '''The function will calculate the missing values and present it in a tabular form '''
  # Calculate the number of missing values in each column
  missing_values_count = dataframe.isnull().sum()

  # Calculate the percentage of missing values in each column
  missing_values_percentage = round(dataframe.isnull().mean().mul(100), 2)

  # Create a table containing the number and percentage of missing values
  missing_values_table = pd.concat([missing_values_count, missing_values_percentage], axis=1)
  missing_values_table = missing_values_table.rename(columns={0: 'Count of Missing Values', 1: '[% of Total Value]'})

  # Sort the table by the percentage of missing values in descending order
  missing_values_table = missing_values_table.sort_values('[% of Total Value]', ascending=False)

  # Reset the index of the table
  missing_values_table = missing_values_table.reset_index().rename(columns={'index': 'Column Name'})

  return missing_values_table


In [13]:
# Missing Values/Null Values Count
missing_values(books)

,Column Name,Count of Missing Values,[% of Total Value]
0,ISBN,0,0.0
1,Book_Title,0,0.0
2,Book_Author,2,0.0
3,Year_Of_Publication,0,0.0
4,Publisher,2,0.0


In [14]:
missing_values(users)

,Column Name,Count of Missing Values,[% of Total Value]
0,Age,110762,39.72
1,User_ID,0,0.00
2,Location,0,0.00


In [15]:
missing_values(ratings)

,Column Name,Count of Missing Values,[% of Total Value]
0,User_ID,0,0.0
1,ISBN,0,0.0
2,Book_Rating,0,0.0


In [16]:
books['Year_Of_Publication'] = pd.to_numeric(books['Year_Of_Publication'], errors='coerce')
users['User_ID'] = users['User_ID'].astype(int)
users['Age'] = users['Age'].astype(float)
ratings['User_ID'] = ratings['User_ID'].astype(int)
ratings['Book_Rating'] = ratings['Book_Rating'].astype(int)

In [17]:
books.loc[(books['Year_Of_Publication'] > 2006) | (books['Year_Of_Publication'] == 0),'Year_Of_Publication'] = np.nan

In [18]:
location = users.Location.str.split(', ', n=2, expand=True)
location.columns=['city', 'state', 'country']

users['city'] = location['city']
users['state'] = location['state']
users['country'] = location['country']

In [19]:
users.head()

,User_ID,Location,Age,city,state,country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom


In [20]:
users['country'].replace(['','01776','02458','19104','23232','30064','85021','87510','alachua','america','austria','autralia','cananda','geermany','italia','united kindgonm','united sates','united staes','united state','united states','us'],
                           ['other','usa','usa','usa','usa','usa','usa','usa','usa','usa','australia','australia','canada','germany','italy','united kingdom','usa','usa','usa','usa','usa'],inplace=True)

C:\Users\hungt\AppData\Local\Temp\ipykernel_15616\3914490809.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  users['country'].replace(['','01776','02458','19104','23232','30064','85021','87510','alachua','america','austria','autralia','cananda','geermany','italia','united kindgonm','united sates','united staes','united state','united states','us'],


In [21]:
users.head()

,User_ID,Location,Age,city,state,country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom


In [22]:
# replace outlier with nan
users.loc[(users.Age > 100) | (users.Age < 5), 'Age'] = np.nan

In [23]:
# filling the Age as per the median value of country
users['Age'] = users['Age'].fillna(users.groupby('country')['Age'].transform('median'))

In [24]:
missing_values(users)

,Column Name,Count of Missing Values,[% of Total Value]
0,country,4577,1.64
1,Age,3081,1.10
2,Location,0,0.00
3,User_ID,0,0.00
4,city,0,0.00
5,state,1,0.00


In [25]:
users['Age'].fillna(users.Age.mean(),inplace=True)

C:\Users\hungt\AppData\Local\Temp\ipykernel_15616\2006163882.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  users['Age'].fillna(users.Age.mean(),inplace=True)


In [26]:
missing_values(users)

,Column Name,Count of Missing Values,[% of Total Value]
0,country,4577,1.64
1,User_ID,0,0.00
2,Location,0,0.00
3,Age,0,0.00
4,city,0,0.00
5,state,1,0.00


In [27]:
# dropping location, city, state columns.
users.drop(['Location','city','state'], axis=1, inplace=True)

In [28]:
# replacing unknown countries with other
users.fillna('others', inplace=True)

In [29]:
missing_values(users)

,Column Name,Count of Missing Values,[% of Total Value]
0,User_ID,0,0.0
1,Age,0,0.0
2,country,0,0.0


**Replaced the outlier (ages less than 5 and more than 99) in the age column with NaN. Replaced the missing age as per the median value of country. Replaced the remaining NaN values with mean age.**

**Replaced the remaining missing values in the users with 'others'**

In [30]:
missing_values(books)

,Column Name,Count of Missing Values,[% of Total Value]
0,Year_Of_Publication,4641,1.71
1,ISBN,0,0.00
2,Book_Title,0,0.00
3,Book_Author,2,0.00
4,Publisher,2,0.00


In [31]:
# replacing missing years with median value
books['Year_Of_Publication'].fillna(round(books['Year_Of_Publication'].median()), inplace=True)


C:\Users\hungt\AppData\Local\Temp\ipykernel_15616\704737649.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books['Year_Of_Publication'].fillna(round(books['Year_Of_Publication'].median()), inplace=True)


In [32]:
missing_values(books)

,Column Name,Count of Missing Values,[% of Total Value]
0,ISBN,0,0.0
1,Book_Title,0,0.0
2,Book_Author,2,0.0
3,Year_Of_Publication,0,0.0
4,Publisher,2,0.0


**Replaced years with median year and replaced remaining missing values with 'others'**

In [33]:
ratings_explicit = ratings[ratings['Book_Rating']!=0]

In [34]:
ratings_explicit

,User_ID,ISBN,Book_Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
...,...,...,...
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [384]:
# # Merging the dataframe using userid and isbn columns
# data = pd.merge(users, ratings_explicit, on = 'User_ID')
# data = pd.merge(data, books, on='ISBN')
# data.head()

In [35]:
# 2. Filter Users: Keep those with > 10 explicit ratings
# Since we removed 0s, a threshold of 10 here is actually harder/better than 50 mixed.
# (You can raise this to 20 or 50 if you have too much data, but 10 is safe)
x = ratings['User_ID'].value_counts() >= 10
y = x[x].index
ratings_explicit = ratings_explicit[ratings_explicit['User_ID'].isin(y)]

# 3. Merge with 'users' (Adds Age/Location)
ratings_explicit = ratings_explicit.merge(users, on='User_ID')

# 4. Merge with 'books' (Adds Title, Author, Year)
ratings_with_name = ratings_explicit.merge(books, on='ISBN')

# 5. Filter Books: Group by Title to handle multiple ISBNs for the same book
# We calculate how many *Explicit* ratings each book title has
number_rating = ratings_with_name.groupby('Book_Title')['Book_Rating'].count().reset_index()
number_rating.rename(columns={'Book_Rating':'number_of_ratings'}, inplace=True)

# Merge the count back
data = ratings_with_name.merge(number_rating, on='Book_Title')

# 6. Final Filter: Keep books with >= 10 ratings
data = data[data['number_of_ratings'] >= 10]

# Check the result
print(f"Final Data Shape: {data.shape}")
print(f"Unique Users: {data['User_ID'].nunique()}")
print(f"Unique Books: {data['Book_Title'].nunique()}")

Final Data Shape: (102793, 10)
Unique Users: 10697
Unique Books: 4106


In [36]:
missing_values(data)

,Column Name,Count of Missing Values,[% of Total Value]
0,User_ID,0,0.0
1,ISBN,0,0.0
2,Book_Rating,0,0.0
3,Age,0,0.0
4,country,0,0.0
5,Book_Title,0,0.0
6,Book_Author,0,0.0
7,Year_Of_Publication,0,0.0
8,Publisher,0,0.0
9,number_of_ratings,0,0.0


In [37]:
data.drop(['number_of_ratings'], axis=1, inplace=True)

In [38]:
data.dtypes

User_ID                  int64
ISBN                    object
Book_Rating              int64
Age                    float64
country                 object
Book_Title              object
Book_Author             object
Year_Of_Publication    float64
Publisher               object
dtype: object

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102793 entries, 0 to 293105
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User_ID              102793 non-null  int64  
 1   ISBN                 102793 non-null  object 
 2   Book_Rating          102793 non-null  int64  
 3   Age                  102793 non-null  float64
 4   country              102793 non-null  object 
 5   Book_Title           102793 non-null  object 
 6   Book_Author          102793 non-null  object 
 7   Year_Of_Publication  102793 non-null  float64
 8   Publisher            102793 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 7.8+ MB


In [40]:
import csv

# 2. Clean hidden newline characters (\n) and carriage returns (\r)
# We replace them with a simple space.
for col in data.columns:
    if col in data.columns:
        # astype(str) ensures we don't crash on numbers
        data[col] = data[col].astype(str).str.replace(r'[\n\r]+', ' ', regex=True)
        # Optional: Unescape HTML like '&amp;' -> '&'
        data[col] = data[col].str.replace('&amp;', '&', regex=False)

# 3. Save with "Safe" Settings
# quoting=csv.QUOTE_ALL puts "quotes" around every field.
# This prevents commas inside book titles (like "Clark, Mary Higgins") from breaking columns.
data.to_csv(
    'book_recomm_dataset.csv', 
    index=False, 
    encoding='utf-8', 
    quoting=csv.QUOTE_ALL, 
    quotechar='"'
)